In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa

import pathlib

from pyvis import network as net



In [2]:
engine = sa.create_engine('sqlite:///mysqlsampledatabase.db', echo=False)
conn = engine.connect()

In [3]:
database_path = pathlib.Path.cwd() / 'data'
for f in database_path.glob('*.csv'):
    table_name = f.stem
    print(table_name)
    table_data = pd.read_csv(f)
    table_data.columns = [c.strip("'") for c in table_data.columns]
    for c in table_data.columns[table_data.dtypes == 'object']:
        table_data[c] = table_data[c].str.strip("'")
    
    #print(table_data.info())
    if table_name in ['orderdetails', 'payments']:
        table_data_indexed = table_data.set_index(table_data.columns[:2].to_list())
    else:
        table_data_indexed = table_data.set_index(table_data.columns[0])
    table_data_indexed.to_sql(table_name, engine, if_exists='replace', index=True)
    table_data_indexed.to_csv(f)

customers
employees
offices
orderdetails
orders
payments
productlines
products


In [4]:
customers = pd.read_sql('SELECT * FROM customers', con=engine)

In [5]:
inspector = sa.inspect(engine)

In [6]:
inspector.get_table_names()

['customers',
 'employees',
 'offices',
 'orderdetails',
 'orders',
 'payments',
 'productlines',
 'products']

In [7]:
employees = pd.read_sql('SELECT * FROM employees', con=engine)

In [8]:
#employees = employees.replace(to_replace={'Sales Manager APAC)': 'Sales Manager (APAC)', 
#                                          'Sale Manager EMEA)': 'Sale Manager (EMEA)', 
#                                          'Sales Manager NA)':'Sales Manager (NA)'})
employees['fullName'] = employees['firstName'] + ' ' + employees['lastName']

In [9]:
jobs = list(employees['jobTitle'].unique())
jobs

['President',
 'VP Sales',
 'VP Marketing',
 'Sales Manager (APAC)',
 'Sale Manager (EMEA)',
 'Sales Manager (NA)',
 'Sales Rep']

In [10]:
employee_name = employees['fullName']
employee_name

0          Diane Murphy
1        Mary Patterson
2         Jeff Firrelli
3     William Patterson
4         Gerard Bondur
5           Anthony Bow
6       Leslie Jennings
7       Leslie Thompson
8        Julie Firrelli
9       Steve Patterson
10       Foon Yue Tseng
11        George Vanauf
12          Loui Bondur
13     Gerard Hernandez
14      Pamela Castillo
15           Larry Bott
16          Barry Jones
17          Andy Fixter
18          Peter Marsh
19             Tom King
20           Mami Nishi
21         Yoshimi Kato
22        Martin Gerard
Name: fullName, dtype: object

In [11]:
employees

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,fullName
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,NaN,President,Diane Murphy
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002.0,VP Sales,Mary Patterson
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002.0,VP Marketing,Jeff Firrelli
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056.0,Sales Manager (APAC),William Patterson
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056.0,Sale Manager (EMEA),Gerard Bondur
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056.0,Sales Manager (NA),Anthony Bow
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143.0,Sales Rep,Leslie Jennings
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143.0,Sales Rep,Leslie Thompson
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143.0,Sales Rep,Julie Firrelli
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143.0,Sales Rep,Steve Patterson


In [12]:
gr = net.Network(notebook=True, cdn_resources='remote', directed =True)
gr.add_nodes(jobs, 
             title=jobs,
             shape=['elipse'] * len(jobs),
             label=jobs)
gr.add_nodes(employee_name.values, 
             title=employee_name,
             shape=['box'] * len(employee_name),
             label=employee_name.values)
for e_name, e_data in employees.set_index('fullName').iterrows():
    if ~np.isnan(e_data['reportsTo']):
        reports_to = employees.loc[employees['employeeNumber']==e_data['reportsTo'], 'fullName'].values[0]
    else:
        reports_to = e_name
    gr.add_edge(e_name, e_data['jobTitle'], label='job_title', physics =True)
    gr.add_edge(e_name, reports_to, label='reports_to')
gr.show('employee_graph.html')

In [13]:
# Once you have finished with the connection, you can close it
conn.close()